In [1]:
# Este entorno de Python 3 es similar a Jupyter Notebook
# Viene con varias librerías instaladas. Para más információn podés consultar 
# la imagen de docker que utiliza (https://github.com/kaggle/docker-python)

import numpy as np 
import pandas as pd
import keras
import matplotlib.pyplot as plt
%matplotlib inline

# Los archivos del zip "aap2020unlp.zip" están disponibles automáticamente en la 
# carpeta "/kaggle/input/aap2020unlp"
# (o "../input/aap2020unlp")
# Por ejemplo, el siguiente código imprime todos los archivos disponibles:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Cualquier resultado que guarden en el directorio actual queda como un "output" en el workspace

In [2]:
from keras.preprocessing.image import load_img

dataset_folderpath = "/kaggle/input/aap2020unlp"
print("Hay 3 directorios con los tres subconjuntos de datos: ",os.listdir(dataset_folderpath))
print("La carpeta de cada subconjunto tiene dos subcarpetas: normal y pneumonia")

train_folderpath = os.path.join(dataset_folderpath, "train")
val_folderpath = os.path.join(dataset_folderpath, "val")
test_folderpath = os.path.join(dataset_folderpath, "test")

img_name = '0000.jpg'
image_path= f'normal/{img_name}'
img_normal = load_img(os.path.join(train_folderpath, image_path))

print(f"Las imágenes tienen tamaño: {img_normal.size}")

plt.imshow(img_normal)
plt.title("Normal")
plt.show()

img_name = '0000.jpg'
image_path= f'pneumonia/{img_name}'
img_pneumonia = load_img(os.path.join(train_folderpath,image_path))
plt.imshow(img_pneumonia)
plt.title("Neumonía")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/aap2020unlp'

In [3]:
from keras.preprocessing.image import ImageDataGenerator
# mismo preprocesamiento que el usado originalmente para entrenar MobileNet
from keras.applications.mobilenet import preprocess_input

# Tamaño objetivo para escalar las imágenes. 
IMG_SIZE = 224
h,w,c = IMG_SIZE, IMG_SIZE, 3
batch_size = 32

# Preprocesamiento de cada subconjunto
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input # mismo preprocesamiento que el usado originalmente para entrenar MobileNet
    )

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input # mismo preprocesamiento que el usado originalmente para entrenar MobileNet
    )

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input # mismo preprocesamiento que el usado originalmente para entrenar MobileNet
    )

# Generadores de los subconjuntos. Reciben un directorio, y 
# cada carpeta del directorio se interpreta como una clase distinta.
# En este caso como cada directorio tiene 3 subdirectorios, normal, covid y pneumonia,
# por ende, habrá tres clases.
train_generator = train_datagen.flow_from_directory(
    train_folderpath, # directorio de donde cargar las imagenes (train)
    target_size=(h, w),
    batch_size=batch_size,
    class_mode='binary')

val_generator = val_datagen.flow_from_directory(
    val_folderpath, # directorio de donde cargar las imagenes (val)
    target_size=(h, w),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_folderpath, # directorio de donde cargar las imagenes (test)
    target_size=(h, w),
    batch_size=batch_size,
    class_mode=None, # IMPORTANTE ya que los ej de test no tienen clase
    shuffle=False # IMPORTANTE ya que nos importa el orden para el archivo de submission
    )


n_train = train_generator.samples
n_val = val_generator.samples
n_test = test_generator.samples
n_clases = train_generator.num_classes
print(f"Los conjuntos de train, val y test tienen {n_train}, {n_val} y {n_test} ejemplos respectivamente.")
print(f"Los conjuntos de datos tienen {n_clases} clases.")

NameError: name 'train_folderpath' is not defined

In [4]:
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.mobilenet import MobileNet

base_model = MobileNet(input_shape=(h, w, c), weights='imagenet', include_top=False) 

for layer in base_model.layers:
    layer.trainable = False # capas “congeladas” no se entrenan

output = GlobalAveragePooling2D()(base_model.output)    
# Utilizar salida del modelo como entrada a capa Dense de 128 
output = Dense(128, activation='relu')(output)
# Nueva capa de salida
output = Dense(1, activation='sigmoid')(output)

# Crear nuevo modelo en base a lo anterior
model = Model(inputs=base_model.input, outputs=output)

model.summary()

17227776/17225924 [==============================] - 1s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_______________________________________________________

In [5]:
from keras import backend as K
# Definición de las métricas F1, recall y precision utilizando Keras.

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
        precision = precision_m(y_true, y_pred)
        recall = recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:
from keras.optimizers import Adam

lr = 1e-3
epochs = 5

optim = Adam(lr=lr, decay=lr / epochs)
val_steps = max(1, n_val // batch_size)

# Compilation with weighted metrics
model.compile(loss="binary_crossentropy", optimizer=optim, metrics=["accuracy", f1_m, precision_m, recall_m])
model.fit_generator(train_generator,
                    steps_per_epoch=n_train // batch_size,
                    epochs=epochs,
                    validation_data=val_generator,
                    validation_steps=val_steps
                    )

NameError: name 'n_val' is not defined

In [7]:
# Evaluar el accuracy del modelo en el conjunto entero de entrenamiento
print("*********** Conjunto de entrenamiento ***********")
train_generator.reset()
_ = model.predict_generator(train_generator, steps=n_train // batch_size)
scores = model.evaluate_generator(train_generator, steps=n_train // batch_size)
for metric, score in zip(model.metrics_names, scores):
    print(f"{metric}: {score:.2f}")

print()
# Evaluar el accuracy del modelo en el conjunto entero de validación
print("*********** Conjunto de validación ***********")
val_generator.reset()
_ = model.predict_generator(val_generator, steps=n_val // batch_size)
scores = model.evaluate_generator(val_generator, steps=n_val // batch_size)
for metric, score in zip(model.metrics_names, scores):
    print(f"{metric}: {score:.2f}")

*********** Conjunto de entrenamiento ***********


NameError: name 'train_generator' is not defined

In [8]:
import csv

# predecir sobre el conjunto de test y generar el csv resultante
y_prob = model.predict_generator(test_generator, steps=n_test // batch_size + 1)
# Establecer un umbral
treshold = 0.5
# Convertir probabilidades a etiquetas con el umbral
y_pred = (y_prob[:,0] > treshold).astype(int)
# quitar el nombre de la carpeta del nombre de archivo
filenames = [os.path.basename(f) for f in test_generator.filenames]
# igual cant de archivos que de predicciones
assert(len(y_pred) == len(filenames))

# Generar CSV con las predicciones
with open('solutions.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer.writerow(['Id', 'Predicted'])
    print("Id, Predicted")
    for f, y in zip(filenames, y_pred):
        print(f"{f}, {str(y)}")
        writer.writerow([f,str(y)])

NameError: name 'test_generator' is not defined